# AutoML Pipeline - Автоматическое машинное обучение

Этот notebook содержит готовые пайплайны для:
- AutoGluon (табличные данные, тексты, изображения)
- PyCaret (альтернативный AutoML)
- H2O AutoML
- Автоматический подбор гиперпараметров

## 1. Установка зависимостей

In [ ]:
# AutoGluon - основной фреймворк
!pip install autogluon -q

# PyCaret - альтернатива
# !pip install pycaret -q

# H2O AutoML - еще одна альтернатива
# !pip install h2o -q

## 2. Импорты

In [ ]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.multimodal import MultiModalPredictor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("Библиотеки загружены!")

## 3. Загрузка данных
**Подставьте свои данные здесь**

In [ ]:
# === ВАШИ ДАННЫЕ ===
# Вариант 1: CSV/TSV файлы
train_df = pd.read_csv('train.csv')  # Замените на свой путь
test_df = pd.read_csv('test.csv')    # Замените на свой путь

# Вариант 2: Использовать встроенные данные AutoGluon для теста
# train_df = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
# test_df = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nПервые строки:\n{train_df.head()}")

## 4. Настройка задачи

In [ ]:
# === НАСТРОЙКИ ===
TARGET_COLUMN = 'target'  # Название целевой колонки
PROBLEM_TYPE = 'binary'   # 'binary', 'multiclass', 'regression'
EVAL_METRIC = 'roc_auc'   # Метрика оценки: 'accuracy', 'f1', 'roc_auc', 'rmse', 'mae' и др.
TIME_LIMIT = 600          # Время обучения в секундах (600 = 10 минут)
PRESETS = 'best_quality'  # 'best_quality', 'high_quality', 'good_quality', 'medium_quality'

# Проверка наличия целевой колонки
if TARGET_COLUMN not in train_df.columns:
    print(f"⚠️ Колонка '{TARGET_COLUMN}' не найдена!")
    print(f"Доступные колонки: {train_df.columns.tolist()}")
else:
    print(f"✓ Целевая колонка: {TARGET_COLUMN}")
    print(f"✓ Тип задачи: {PROBLEM_TYPE}")
    print(f"✓ Метрика: {EVAL_METRIC}")

## 5. Обучение модели (AutoGluon Tabular)

In [ ]:
# Создание предиктора
predictor = TabularPredictor(
    label=TARGET_COLUMN,
    problem_type=PROBLEM_TYPE,
    eval_metric=EVAL_METRIC,
    path='./AutogluonModels'  # Путь для сохранения моделей
)

# Обучение
predictor.fit(
    train_data=train_df,
    time_limit=TIME_LIMIT,
    presets=PRESETS,
    # Дополнительные параметры:
    # num_bag_folds=5,        # Количество фолдов для бэггинга
    # num_bag_sets=1,         # Количество наборов бэггинга
    # num_stack_levels=1,     # Количество уровней стекинга
)

print("\n✓ Обучение завершено!")

## 6. Анализ результатов

In [ ]:
# Таблица лидеров
leaderboard = predictor.leaderboard(silent=True)
print("Топ-10 моделей:")
print(leaderboard.head(10))

# Лучшая модель
print(f"\nЛучшая модель: {predictor.get_model_best()}")

In [ ]:
# Важность признаков
feature_importance = predictor.feature_importance(train_df)
print("\nТоп-15 важных признаков:")
print(feature_importance.head(15))

## 7. Предсказания и сабмит

In [ ]:
# Предсказания на тесте
if PROBLEM_TYPE in ['binary', 'multiclass']:
    # Вероятности классов
    predictions_proba = predictor.predict_proba(test_df)
    # Классы
    predictions = predictor.predict(test_df)
    print(f"Предсказания (первые 5): {predictions[:5]}")
else:
    # Регрессия
    predictions = predictor.predict(test_df)
    print(f"Предсказания (первые 5): {predictions[:5]}")

print(f"\nВсего предсказаний: {len(predictions)}")

In [ ]:
# === СОЗДАНИЕ SUBMISSION ===
# Формат зависит от задачи

# Для классификации с вероятностями:
if PROBLEM_TYPE in ['binary', 'multiclass']:
    submission = pd.DataFrame({
        'id': test_df.index if 'id' not in test_df.columns else test_df['id'],
        'prediction': predictions
    })
    # Или с вероятностями:
    # submission = pd.DataFrame({
    #     'id': test_df['id'],
    #     'probability': predictions_proba.iloc[:, 1]  # Вероятность положительного класса
    # })
else:
    # Для регрессии:
    submission = pd.DataFrame({
        'id': test_df.index if 'id' not in test_df.columns else test_df['id'],
        'prediction': predictions
    })

submission.to_csv('automl_submission.csv', index=False)
print("✓ Submission сохранен: automl_submission.csv")
print(submission.head())

## 8. Валидация (опционально)

In [ ]:
# Если есть валидационная выборка или хотите сделать split
train_data, val_data = train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=42,
    stratify=train_df[TARGET_COLUMN] if PROBLEM_TYPE != 'regression' else None
)

# Обучение с валидацией
predictor_val = TabularPredictor(
    label=TARGET_COLUMN,
    problem_type=PROBLEM_TYPE,
    eval_metric=EVAL_METRIC,
    path='./AutogluonModels_val'
)

predictor_val.fit(
    train_data=train_data,
    tuning_data=val_data,  # Валидационные данные
    time_limit=TIME_LIMIT,
    presets=PRESETS,
)

# Оценка на валидации
val_score = predictor_val.evaluate(val_data)
print(f"\nОценка на валидации: {val_score}")

## 9. Продвинутые настройки

In [ ]:
# Настройка конкретных моделей
hyperparameters = {
    'GBM': [  # LightGBM
        {'num_leaves': 30, 'learning_rate': 0.03},
        {'num_leaves': 50, 'learning_rate': 0.05},
    ],
    'CAT': {},  # CatBoost (дефолтные параметры)
    'XGB': {},  # XGBoost
    'RF': [   # Random Forest
        {'n_estimators': 100},
    ],
    'NN_TORCH': {},  # Нейронная сеть
}

predictor_advanced = TabularPredictor(
    label=TARGET_COLUMN,
    problem_type=PROBLEM_TYPE,
    eval_metric=EVAL_METRIC,
    path='./AutogluonModels_advanced'
)

predictor_advanced.fit(
    train_data=train_df,
    hyperparameters=hyperparameters,
    time_limit=TIME_LIMIT,
    num_bag_folds=5,
    num_stack_levels=2,  # Двухуровневый стекинг
)

print("\n✓ Продвинутая модель обучена!")

## 10. Multimodal данные (текст + табличные)

In [ ]:
# Если в данных есть текстовые колонки вместе с числовыми
# AutoGluon автоматически обработает разные типы данных

# Пример с текстовой колонкой
# train_df['text_column'] = "Какой-то текст..."

# multimodal_predictor = MultiModalPredictor(
#     label=TARGET_COLUMN,
#     problem_type=PROBLEM_TYPE,
#     eval_metric=EVAL_METRIC,
#     path='./MultiModalModels'
# )

# multimodal_predictor.fit(
#     train_data=train_df,
#     time_limit=TIME_LIMIT,
# )

# predictions_mm = multimodal_predictor.predict(test_df)
# print("Multimodal предсказания готовы!")

## 11. PyCaret (альтернативный AutoML)

In [ ]:
# from pycaret.classification import *  # Для классификации
# # from pycaret.regression import *    # Для регрессии

# # Настройка окружения
# clf = setup(
#     data=train_df,
#     target=TARGET_COLUMN,
#     session_id=42,
#     silent=True,
#     verbose=False
# )

# # Сравнение моделей
# best_model = compare_models(n_select=3, sort=EVAL_METRIC)

# # Тюнинг лучшей модели
# tuned_model = tune_model(best_model)

# # Предсказания
# predictions_pycaret = predict_model(tuned_model, data=test_df)
# print("PyCaret предсказания готовы!")